1 - get the nodes state observations : their informations through the rounds + the accuracy of their models

In [4]:
import json

# Load the JSON data from the file
with open('../../../blockchain.json', 'r') as f:
    data = json.load(f)

# Create a list to store node information for each block
node_info_by_block = {}

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']

    for transaction in json_obj['Transactions']:
        content = transaction.get('Content', {})
        message_type = content.get('message_type')
        node_id = content.get('nodeId')

        if message_type == 1 and node_id is not None:
            availability = int(content.get('availability'))
            honesty = content.get('honesty')
            dataset_size = content.get('datasetSize')
            freq = content.get('freq')
            trans_rate = content.get('transRate')
            task = content.get('task')
            dropout = content.get('dropout', 0.0)

            # Find accuracy for the corresponding node in message_type 2 transactions
            accuracy = 0.0
            for json_obj2 in data:
                if json_obj2['BlockId'] == block_id:
                    for transaction2 in json_obj2['Transactions']:
                        if transaction2['Content'].get('message_type') == 2 and transaction2['Content'].get('nodeId') == node_id:
                            accuracy = transaction2['Content'].get('accuracy', 0.0)
                            break
                    if accuracy != 0.0:
                        break

            node_info = [node_id, availability, honesty, dataset_size, freq, trans_rate, task, dropout, accuracy]

            if block_id not in node_info_by_block:
                node_info_by_block[block_id] = []

            node_info_by_block[block_id].append(node_info)

# Sort the node lists within each block by nodeId
for nodes in node_info_by_block.values():
    nodes.sort(key=lambda x: x[0])  # Sort based on nodeId
all_nodes = []
[all_nodes.append(nodes) for nodes in node_info_by_block.values()]
print(all_nodes[0])
# # Print or use node_info_by_block as needed
# for block_id, nodes in node_info_by_block.items():
#     print(f"Block {block_id}:")
#     for node_info in nodes:
#         print(node_info)
   


[[0, 1, 0.0, 160, 80.0, 460.0, 0, 0.0, 46.25], [1, 1, 0.0, 560, 110.0, 210.0, 1, 0.0, 0.0], [2, 1, 0.0, 460, 60.0, 300.0, 0, 0.0, 0.0], [3, 1, 0.0, 470, 170.0, 200.0, 1, 0.0, 0.0], [4, 1, 0.0, 730, 160.0, 450.0, 0, 0.0, 62.33], [5, 1, 0.0, 130, 300.0, 990.0, 1, 0.0, 50.0], [6, 1, 0.0, 850, 220.0, 420.0, 0, 0.0, 0.0], [7, 1, 0.0, 630, 190.0, 390.0, 0, 0.0, 0.0], [8, 1, 0.0, 300, 90.0, 350.0, 0, 0.0, 0.0], [9, 0, 0.0, 660, 230.0, 950.0, 0, 0.0, 0.0], [10, 0, 0.0, 660, 300.0, 690.0, 0, 0.0, 0.0], [11, 1, 0.0, 880, 250.0, 990.0, 0, 0.0, 0.0], [12, 1, 0.0, 200, 170.0, 210.0, 0, 0.0, 0.0], [13, 1, 0.0, 540, 100.0, 800.0, 0, 0.0, 57.59], [14, 1, 0.0, 580, 180.0, 400.0, 0, 0.0, 54.48], [15, 1, 0.0, 840, 160.0, 260.0, 1, 0.0, 0.0], [16, 0, 0.0, 740, 80.0, 600.0, 0, 0.0, 0.0], [17, 1, 0.0, 1000, 100.0, 490.0, 0, 0.0, 0.0], [18, 1, 0.0, 480, 60.0, 830.0, 0, 0.0, 0.0], [19, 1, 0.0, 420, 50.0, 940.0, 0, 0.0, 0.0], [20, 1, 0.0, 640, 130.0, 520.0, 0, 0.0, 53.12], [21, 1, 0.0, 300, 70.0, 930.0, 0, 0.0

2 - get the global accuracy for each round

In [11]:
# Create a list to store the global accuracy values
global_accuracy = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 2 and type equal to 2
    accuracy_transaction = None
    for transaction in json_obj['Transactions']:
        content = transaction['Content']
        if content.get('message_type') == 2 and content.get('type') == 2:
            accuracy_transaction = transaction
            break
    
    if accuracy_transaction is not None:
        accuracy = accuracy_transaction['Content'].get('accuracy', 0.0)
        global_accuracy.append(accuracy)

# Print or use global_accuracy as needed
print("Global Accuracy Values:", global_accuracy)

Global Accuracy Values: [63.08, 77.69, 81.76, 79.82]


In [7]:
# Create a list to store the concatenated lists for each block
selected_nodes = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes.append(aggregators + trainers)

# Print or use selected_nodes as needed
for block_id, selected_node_list in zip([json_obj['BlockId'] for json_obj in data], selected_nodes):
    print(f"Block {block_id}: {selected_node_list}")

Block 1/0: [35, 82, 79, 17, 48, 49, 88, 30, 15, 90, 65, 83, 57, 3, 1, 0, 54, 97, 85, 81, 21, 93, 56, 64, 2, 41, 60, 91, 22, 55, 74, 33, 62, 94, 63, 16, 39, 18, 76, 86, 38, 26, 98, 14, 67, 59, 66, 36, 51, 5]
Block 1/1: [35, 79, 90, 30, 17, 78, 15, 83, 45, 68, 1, 3, 81, 40, 71, 85, 97, 54, 21, 56, 64, 41, 91, 69, 22, 55, 33, 87, 74, 18, 76, 39, 86, 7, 38, 61, 11, 72, 66, 98, 67, 4, 59, 36, 73, 51, 5, 25, 96, 23]
Block 1/2: [90, 81, 79, 83, 1, 78, 49, 3, 85, 68, 97, 65, 57, 88, 56, 54, 9, 21, 0, 64, 45, 41, 40, 91, 34, 22, 55, 69, 87, 20, 15, 74, 86, 7, 39, 61, 38, 94, 63, 66, 72, 67, 59, 5, 51, 26, 96, 23, 25, 53]
Block 1/3: [81, 85, 1, 90, 3, 49, 78, 97, 35, 0, 65, 30, 21, 48, 64, 82, 54, 9, 71, 68, 45, 41, 34, 40, 2, 60, 69, 17, 87, 86, 20, 74, 76, 38, 72, 94, 59, 67, 63, 51, 26, 36, 73, 98, 96, 23, 25, 53, 15, 31]


In [8]:
from collections import defaultdict


# Create a dictionary to store the concatenated lists for each block
selected_nodes = defaultdict(list)

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes[block_id] = aggregators + trainers

# Count how many times each node was selected
node_selection_counts = defaultdict(int)
for selected_node_list in selected_nodes.values():
    for node_id in selected_node_list:
        node_selection_counts[node_id] += 1

# Print or use node_selection_counts as needed
for node_id, count in node_selection_counts.items():
    print(f"Node {node_id} was selected {count} times.")


Node 35 was selected 3 times.
Node 82 was selected 2 times.
Node 79 was selected 3 times.
Node 17 was selected 3 times.
Node 48 was selected 2 times.
Node 49 was selected 3 times.
Node 88 was selected 2 times.
Node 30 was selected 3 times.
Node 15 was selected 4 times.
Node 90 was selected 4 times.
Node 65 was selected 3 times.
Node 83 was selected 3 times.
Node 57 was selected 2 times.
Node 3 was selected 4 times.
Node 1 was selected 4 times.
Node 0 was selected 3 times.
Node 54 was selected 4 times.
Node 97 was selected 4 times.
Node 85 was selected 4 times.
Node 81 was selected 4 times.
Node 21 was selected 4 times.
Node 93 was selected 1 times.
Node 56 was selected 3 times.
Node 64 was selected 4 times.
Node 2 was selected 2 times.
Node 41 was selected 4 times.
Node 60 was selected 2 times.
Node 91 was selected 3 times.
Node 22 was selected 3 times.
Node 55 was selected 3 times.
Node 74 was selected 4 times.
Node 33 was selected 2 times.
Node 62 was selected 1 times.
Node 94 was se